# Comparing a USGSCSM and ISIS camera for Kaguya Terrain Camera

In [1]:
import pvl
import os
import csmapi
import json

import numpy as np

import ale
import csmapi

os.environ['ISISROOT'] = '/usgs/cpkgs/anaconda3_linux/envs/isis3.9.0'
import knoten
from knoten import vis
from pysis import isis
from pysis.exceptions import ProcessError

import ctypes
from ctypes.util import find_library

# Register the usgscam plugin with the csmapi
libusgscsm_path = find_library('usgscsm')

if not libusgscsm_path:
    warnings.warn('libusgscsm not installed, unable to load shared library.')

libusgscsm = ctypes.CDLL(libusgscsm_path)

if not libusgscsm._name:
    warnings.warn('Unable to load usgscsm shared library')

/home/acpaquette/repos/ale/ale/__init__.py:22: UserWarning: ALESPICEROOT environment variable not set, Spice Drivers will not function correctly
  warnings.warn('ALESPICEROOT environment variable not set, Spice Drivers will not function correctly')


## Make a CSM sensor model
Requires TC1S2B0_01_02842S506E1942.img in data directory

In [39]:
fileName = 'data/TC1S2B0_01_02842S506E1942.img'

## Ingest the image and spiceinit

In [92]:
json_file = os.path.splitext(fileName)[0] + '.json'
cub_loc = os.path.splitext(fileName)[0] + '.cub'

try: 
    isis.kaguyatc2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

try:
    isis.spiceinit(from_=cub_loc, shape='ellipsoid', iak='/home/arsanders/testData/kaguyaTcAddendum007.ti')
except ProcessError as e:
    print(e.stderr)
    
kernels = ale.util.generate_kernels_from_cube(cub_loc, expand=True)
usgscsm_str = ale.loads(fileName, props={'kernels': kernels}, formatter="usgscsm", verbose=False)

# # ale.load(fileName, props={'kernels': kernels}, formatter="usgscsm", verbose=False)

with ale.drivers.selene_drivers.KaguyaTcPds3NaifSpiceDriver(fileName, props={'kernels': kernels}) as driver:
    print(driver.exposure_duration)
    start_time = driver.ephemeris_start_time
    print(start_time)
    scan_rate = driver.line_scan_rate[-1][0]
#     print(scan_rate * 2)
    for i in range(driver.image_lines):
        start_time += scan_rate
    print(start_time)
    print(driver.ephemeris_stop_time)
        

with open(json_file, 'w') as isd_file:
    usgscsm_json = json.loads(usgscsm_str)
#     usgscsm_json['detector_center'] = {'line': 0.5, 'sample': 2048.0}
    usgscsm_json['focal2pixel_lines'] = [0, 142.85714285714286, 0]
    usgscsm_json['optical_distortion']['kaguyalism'] = {'x': [0, 0, 0, 0], 'y': [0, 0, 0, 0], 'boresight_x': 0, 'boresight_y': 0}
    usgscsm_json['log_file'] = '/home/acpaquette/logs/usgscsm_output.log'
#     usgscsm_json['optical_distortion']['kaguyalism'] = {'x': [-0.0009649900000000001, 0.00098441, 8.5773e-06, -3.7438e-06], 'y': [-0.0013796, 1.3502e-05, 2.7251e-06, -6.193800000000001e-06], 'boresight_x': -0.0725, 'boresight_y': 0.0214}
#     usgscsm_json['optical_distortion']['kaguyalism'] = {'x': [-0.0009649900000000001, 0.00098441, 8.5773e-06, -3.7438e-06], 'y': [-0.0013796, 1.3502e-05, 2.7251e-06, -6.193800000000001e-06], 'boresight_x': .5, 'boresight_y': 2048}
# #     usgscsm_json['optical_distortion']['kaguyalism'] = {'x': [0, 0, 0, 0], 'y': [0, 0, 0, 0], 'boresight_x': -0.0725, 'boresight_y': 0.0214}
#     print(usgscsm_json['optical_distortion']['kaguyalism'])
#     print(usgscsm_json['focal2pixel_lines'])
#     print(usgscsm_json['focal2pixel_samples'])
    usgscsm_str = json.dumps(usgscsm_json)
    isd_file.write(usgscsm_str)
    
camera = knoten.csm.create_csm(fileName)

0.006499965000000001
265300597.96209636
265300628.22598425
265300628.2259334


In [94]:
image_size = camera.getImageSize()
line, sample = (image_size.line/2), (image_size.samp/2)
print(line, sample)

try:
    pvlres = isis.campt(from_=cub_loc, line=line + .5, sample=sample + .5)
except ProcessError as e:
    print(e.stderr)
    
pvlres = pvl.load(pvlres)['GroundPoint']
pvlres["Sample"] -= .5
pvlres["Line"] -= .5

print(np.array(pvlres['BodyFixedCoordinate'].value) * 1000)

image_coord = csmapi.ImageCoord(line, sample)
ground_coord = camera.imageToGround(image_coord, 0.0)

import pyproj

semi_major, semi_minor = knoten.csm.get_radii(camera)
source_pyproj = pyproj.Proj(proj = 'geocent', a = semi_major, b = semi_minor)
dest_pyproj = pyproj.Proj(proj = 'latlong', a = semi_major, b = semi_minor)

lon, lat, alt = pyproj.transform(source_pyproj, dest_pyproj, ground_coord.x, ground_coord.y, ground_coord.z)
lon = (lon + 360) % 360

try:
    pvlres = isis.campt(from_=cub_loc, lat=lat, lon=lon, type='ground')
except ProcessError as e:
    print(e.stderr)
    
pvlres = pvl.load(pvlres)['GroundPoint']
pvlres["Sample"] -= .5
pvlres["Line"] -= .5
print(pvlres["Line"], pvlres["Sample"])

print(ground_coord.x, ground_coord.y, ground_coord.z)
final_image_coord = camera.groundToImage(ground_coord)
final_image_coord.line, final_image_coord.samp

2328.0 872.0
[-1068277.3256248   -270669.98959043 -1343160.478979  ]
2327.6819020122 872.30170860232
-1068279.0882062048 -270673.1774884656 -1343158.4346939928


(2327.9999974760212, 872.0000000385417)

In [95]:
locus = camera.imageToRemoteImagingLocus(image_coord)
locus.direction.x, locus.direction.y, locus.direction.z

(0.7918331413891762, 0.2059819344083807, 0.5749536667377041)

In [96]:
line = 1000
sample = 0

try:
    pvl_output = pvl.loads(isis.campt(from_=cub_loc, line=line + .5, sample= + .5))['GroundPoint']
    print(pvl_output['EphemerisTime'].value)
except ProcessError as e:
    print(e.stderr)

csm_time = camera.getImageTime(csmapi.ImageCoord(line, sample))
round(usgscsm_json['center_ephemeris_time'] + csm_time, 5)

265300604.46206


265300604.46206

## Compare USGS CSM and ISIS pixels

In [97]:
csmisis_diff_lv_plot, csmisis_diff_ephem_plot, external_orientation_data = vis.external_orientation_diff(json_file, cub_loc, 10, 50, 600, 600)

In [98]:
csmisis_diff_ephem_plot

In [99]:
csmisis_diff_lv_plot

In [100]:
external_orientation_data.head(10)

,isis pos x,isis pos y,isis pos z,csm lv x,csm lv y,csm lv z,isis lv x,isis lv y,isis lv z,diffx,diffy,diffz,diffu,diffv,diffw,xyz_magnitude,uvw_magnitude,isis ephem time,csm ephem time,diff ephem
0,-1.157136e+06,-294123.981512,-1.378203e+06,0.774930,0.288434,0.562396,0.774986,0.288462,0.562304,2.002344e-08,0.000000,-2.002344e-08,-0.000056,-0.000028,0.000092,2.831741e-08,0.000111,2.653006e+08,2.653006e+08,0.0
1,-1.156414e+06,-293922.739154,-1.378860e+06,0.774639,0.288356,0.562837,0.774695,0.288385,0.562745,-9.688921e-05,0.000020,-6.349082e-05,-0.000056,-0.000028,0.000092,1.175510e-04,0.000111,2.653006e+08,2.653006e+08,0.0
2,-1.155692e+06,-293721.412078,-1.379516e+06,0.774347,0.288279,0.563278,0.774404,0.288307,0.563186,-2.023622e-04,0.000030,-9.702332e-05,-0.000056,-0.000028,0.000092,2.263950e-04,0.000111,2.653006e+08,2.653006e+08,0.0
3,-1.154969e+06,-293520.000337,-1.380172e+06,0.774051,0.288210,0.563720,0.774107,0.288238,0.563628,-2.913093e-04,0.000038,-1.234624e-04,-0.000056,-0.000028,0.000092,3.186224e-04,0.000111,2.653006e+08,2.653006e+08,0.0
4,-1.154247e+06,-293318.504013,-1.380828e+06,0.773755,0.288141,0.564162,0.773811,0.288170,0.564070,-3.137279e-04,0.000056,-1.884056e-04,-0.000056,-0.000028,0.000092,3.702875e-04,0.000111,2.653006e+08,2.653006e+08,0.0
5,-1.153524e+06,-293116.923157,-1.381483e+06,0.773458,0.288073,0.564603,0.773514,0.288101,0.564512,-3.588113e-04,0.000063,-2.112952e-04,-0.000056,-0.000028,0.000092,4.211315e-04,0.000111,2.653006e+08,2.653006e+08,0.0
6,-1.152800e+06,-292915.257831,-1.382138e+06,0.773161,0.287999,0.565048,0.773217,0.288027,0.564956,-3.418385e-04,0.000081,-2.698912e-04,-0.000056,-0.000028,0.000091,4.429738e-04,0.000111,2.653006e+08,2.653006e+08,0.0
7,-1.152076e+06,-292713.508087,-1.382793e+06,0.772863,0.287923,0.565493,0.772920,0.287951,0.565402,-3.467270e-04,0.000087,-2.888269e-04,-0.000056,-0.000028,0.000091,4.596324e-04,0.000111,2.653006e+08,2.653006e+08,0.0
8,-1.151352e+06,-292511.674007,-1.383447e+06,0.772566,0.287847,0.565939,0.772622,0.287875,0.565847,-3.118936e-04,0.000101,-3.250279e-04,-0.000056,-0.000028,0.000091,4.615536e-04,0.000111,2.653006e+08,2.653006e+08,0.0
9,-1.150628e+06,-292309.755642,-1.384100e+06,0.772266,0.287772,0.566385,0.772323,0.287801,0.566294,-2.772289e-04,0.000109,-3.438594e-04,-0.000057,-0.000028,0.000091,4.549942e-04,0.000111,2.653006e+08,2.653006e+08,0.0


In [101]:
isis2csm_plot, csm2isis_plot, isiscsm_plotlatlon, isiscsm_plotbf, isis2csm_data, csm2isis_data, isiscsm_latlondata, isiscsm_bfdata = vis.reprojection_diff(json_file, cub_loc, 10, 50, 500, 500)

In [102]:
isis2csm_data[['diff line', 'diff sample']].describe()

,diff line,diff sample
count,500.000000,500.000000
mean,-0.680545,0.261073
std,0.208543,0.050540
min,-1.009768,0.140166
25%,-0.887970,0.231943
50%,-0.689682,0.282553
75%,-0.472854,0.300891
max,-0.319572,0.308153


In [103]:
csm2isis_data[['diff line', 'diff sample']].describe()

,diff line,diff sample
count,500.000000,500.000000
mean,-86.108109,1.135545
std,615.489845,45.472466
min,-4562.559230,-871.500000
25%,0.472394,-0.300883
50%,0.623136,-0.281392
75%,0.888026,-0.231469
max,1.009509,174.254580


In [104]:
isis2csm_plot

In [105]:
csm2isis_plot

In [106]:
csm2isis_data.describe()

,csm sample,csm line,isis sample,isis line,diff sample,diff line,magnitude,angles
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,784.800000,2281.440000,783.664455,2367.548109,1.135545,-86.108109,87.874662,1.908452
std,501.427541,1345.148407,500.220326,1338.450884,45.472466,615.489845,616.918812,0.528124
min,0.000000,0.000000,0.140688,92.116435,-871.500000,-4562.559230,0.436444,-1.929073
25%,348.800000,1117.440000,349.079790,1209.903774,-0.300883,0.472394,0.562446,1.824261
50%,784.800000,2281.440000,784.704061,2327.610094,-0.281392,0.623136,0.798324,1.923669
75%,1220.800000,3445.440000,1221.088650,3537.797491,-0.231469,0.888026,0.910452,2.142565
max,1569.600000,4562.880000,1569.801984,4562.559230,174.254580,1.009509,4565.879703,2.330175


In [107]:
isis2csm_data.describe()

,csm sample,csm line,isis sample,isis line,diff sample,diff line,magnitude,angles
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,784.538927,2282.120545,784.800000,2281.440000,0.261073,-0.680545,0.739060,-1.162159
std,501.415661,1345.149110,501.427541,1345.148407,0.050540,0.208543,0.176368,0.177446
min,-0.151090,0.320589,0.000000,0.000000,0.140166,-1.009768,0.436561,-1.432240
25%,348.520275,1118.162442,348.800000,1117.440000,0.231943,-0.887970,0.562620,-1.316711
50%,784.498483,2282.103552,784.800000,2281.440000,0.282553,-0.689682,0.745279,-1.178452
75%,1220.511197,3446.095871,1220.800000,3445.440000,0.300891,-0.472854,0.909975,-0.998891
max,1569.408600,4563.887668,1569.600000,4562.880000,0.308153,-0.319572,1.020519,-0.811779
